In [147]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create("es0")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [148]:
es.index(index='es0', body={"content": "this",
                            "name": "that"})

es.index(index='es0', body={"content": "that",
                            "name": "this"})

es.index(index='es0', body={"content": "fuck",
                            "name": "that this"})

es.index(index='es0', body={"content": "ca",
                            "name": "this that"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': 'r8npNIYBw-CyhYObGcDL',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

In [149]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            "fields": ["content", "name"],
            "query": "this AND that",
            # 参数含义参考`multi_match`
            "type": "best_fields"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.5753642,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'rMnpNIYBw-CyhYObGcBv',
    '_score': 0.5753642,
    '_source': {'content': 'this', 'name': 'that'}}]}}

In [150]:
'''
where the status field contains active
    status:active

where the title field contains quick or brown
    title:(quick OR brown)

where the author field contains the exact phrase "john smith"
    author:"John Smith"

where the first name field contains Alice (note how we need to escape the space with a backslash)
    first\ name:Alice

where any of the fields book.title, book.content or book.date contains quick or brown (note how we need to escape the * with a backslash):
    book.\*:(quick OR brown)
'''
es.search(index='es0', body=
{
    "query": {
        "query_string": {
            # 匹配:{'content': 'this', 'name': 'that'}}
            # 匹配:{'content': 'fuck', 'name': 'that this'}
            # 不匹配:{'content': 'that', 'name': 'this'}
            "query": "(content:this OR name:this) AND (name:that)"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 1.6169648,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'rMnpNIYBw-CyhYObGcBv',
    '_score': 1.6169648,
    '_source': {'content': 'this', 'name': 'that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'rsnpNIYBw-CyhYObGcDD',
    '_score': 0.62774795,
    '_source': {'content': 'fuck', 'name': 'that this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'r8npNIYBw-CyhYObGcDL',
    '_score': 0.62774795,
    '_source': {'content': 'ca', 'name': 'this that'}}]}}

In [151]:
es.search(index='es0', body=
{
    "query": {
        "query_string": {
            # 匹配:{'content': 'fuck', 'name': 'that this'}
            # 不匹配:{'content': 'fuck', 'name': 'this that'}
            "query": 'name:"that this"'
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.62774795,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'rsnpNIYBw-CyhYObGcDD',
    '_score': 0.62774795,
    '_source': {'content': 'fuck', 'name': 'that this'}}]}}

In [152]:
es.indices.delete('es0')

{'acknowledged': True}